In [46]:
import os
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
%pylab inline
import matplotlib.image as mpimg
import re
import cv2

Populating the interactive namespace from numpy and matplotlib


In [58]:
# Get lists of images and masks
image_path = '/Users/Claude/Documents/GitHub/DSCI591-Fall21-RecommendationSystem/data/image_data'
image_files = []
for root, dirs, files in os.walk(image_path):
    image_files.append(files)
    break
mask_files = []
mask_path = '/Users/Claude/Documents/GitHub/DSCI591-Fall21-RecommendationSystem/data/masks'
for root, dirs, files in os.walk(mask_path):
    mask_files.append(files)
    break

In [59]:
# Flatten the lists
image_files = [image for sublist in image_files for image in sublist]
mask_files = [mask for sublist in mask_files for mask in sublist]

In [60]:
result_files = []
result_path = '/Users/Claude/Documents/GitHub/DSCI591-Fall21-RecommendationSystem/data/masked_image'
for root, dirs, files in os.walk(result_path):
    result_files.append(files)
    break
result_files = [result for sublist in result_files for result in sublist]

In [61]:
len(result_files)

1057

In [62]:
len(image_files)

2372

In [63]:
len(mask_files)

1091

So there are 2372 raw images, 1091 masks generated, 1059 pairs??? and 1057 results.

In [64]:
# Pair the image with cooresponding mask
num_pattern = '(\d*)'

unpaired_mask = []

mask_image_pairs = {}
for mask_i in mask_files:
    mask_ID = re.match(num_pattern, mask_i).group()
    
    for image_i in image_files:
        image_ID = re.match(num_pattern, image_i).group()
        if mask_ID == image_ID:
            mask_image_pairs[image_ID] = (image_i, mask_i)
            mask_files.remove(mask_i)
            break


In [65]:
len(mask_files)

545

In [56]:
k = len(mask_image_pairs)
k

1059

In [39]:
unpaired_mask = sorted(unpaired_mask)
len(unpaired_mask)

1059

In [57]:
# Iterate through the image and mask pairs and combine them together
k = len(mask_image_pairs)
with tqdm(total=k) as pbar:
    for i in mask_image_pairs:
        pbar.update(1)
        image_i, mask_i = mask_image_pairs[i]
        try:
            mask_image(image_i, mask_i, image_path, mask_path)
        except Exception as e:
            print(e)
            print(image_i, mask_i)
            
            

    
# #load examples

# image = plt.imread(os.path.join(image_path, image_i), 0)

 21%|██▏       | 227/1059 [00:24<01:46,  7.81it/s]

Inconsistent shape between the condition and the input (got (3000, 2000) and (1500, 1000))
1902_Indigo (blue)  Dye Kit + Good Tee Unisex_white_thegoodtee.png 1902_0_MASK.pkl


 71%|███████   | 750/1059 [01:21<00:46,  6.68it/s]

Inconsistent shape between the condition and the input (got (1200, 960) and (3000, 2000))
1803_Freedom Long Sleeve Tee_Black_thegoodtee.png 1803_0_MASK.pkl


100%|██████████| 1059/1059 [01:56<00:00,  9.06it/s]


In [27]:
def mask_image(image_i, mask_i, image_path, mask_path):

    # Load image
    image = plt.imread(os.path.join(image_path, image_i), 0)

    # Load mask
    with open(mask_path+'/'+mask_i, 'rb') as file:
        mask_dict = pickle.load(file)

    #separate into r, g, b instances and mask each one
    r, g, b = np.ma.masked_where(mask_dict[0]['masks'][:,:,0]==0, image[:,:,0]), np.ma.masked_where(mask_dict[0]['masks'][:,:,0]==0, image[:,:,1]), np.ma.masked_where(mask_dict[0]['masks'][:,:,0]==0, image[:,:,2])

    #create initial blank mask 
    mask = np.zeros(image.shape, dtype=np.uint8)
    mask = cv2.circle(mask, (260, 300), 225, (255,255,255), -1) 

    # Mask input image with binary mask
    result = cv2.bitwise_and(image, mask)

    # Color background white
    result[mask==0] = 255 # Optional

    # image np array applied with mask
    img = result * ~r.mask[..., None]

    # Use mask_ID as unique ID for each file
    num_pattern = "(\d*)"
    file_ID = re.match(num_pattern, mask_i).group()
    folder_address = "/Users/Claude/Documents/GitHub/DSCI591-Fall21-RecommendationSystem/data/masked_image"
    file_name = "masked_" + file_ID + ".png"

    # Save the output in the data/masked_image
    im = Image.fromarray(img)
    im.save(os.path.join(folder_address, file_name))